In [9]:
import cv2 
import numpy as np
import pytesseract
from pytesseract import Output
import statistics
import json

In [10]:
def getFontSize(d):
    """s = 0
    num = 0
    a = []
    for i in d['text']:
        if i.isspace() or i == "":
            num = num + 1
            continue
        else:
            s = s + d["height"][num]
            a.append(d["height"][num])
            num = num + 1
    avg = s/len(d["text"])"""
    
    return int(statistics.median(d["height"]))
    

In [11]:
def getKernel(size):
    x = np.ones((size,size))
    return x

In [ ]:
rows_list = []
#reading image to array
image = cv2.imread('/home/cp/invoice_images/4/101396.jpg',0) 
#converting image to data
d = pytesseract.image_to_data(image, output_type=Output.DICT)
#getting average font size of the document
font = getFontSize(d)
#print(font)
#print(font)
#calculating approx line space of the document
line_space = int(1.4*font)
#initialising the kernel
kernel = getKernel(int(line_space))

#adjusting the image shape to fit kernel sliding
height,width = image.shape[0],image.shape[1]
pad_h = line_space - (height % line_space)
if pad_h > 1:
    pad_h_1 = int(pad_h/2)
    pad_h_2 = pad_h - pad_h_1
    top_pad_arr_1 = np.full((pad_h_1,width),255)
    top_pad_arr_2 = np.full((pad_h_2,width),255)
    image = np.append(top_pad_arr_1,image,axis=0)
    image = np.append(image,top_pad_arr_2,axis=0)
else:
    top_pad_arr = np.full((pad_h,width),255)
    image = np.append(image,top_pad_arr,axis = 0)
pad_w = line_space - (width % line_space)
if pad_w > 1:
    pad_w_1 = int(pad_w/2)
    pad_w_2 = pad_w - pad_w_1
    bot_pad_arr_1 = np.full((height+pad_h,pad_w_1),255)
    bot_pad_arr_2 = np.full((height+pad_h,pad_w_2),255)
    image = np.append(bot_pad_arr_1,image,axis=1)
    image = np.append(image,bot_pad_arr_2,axis=1)
else:
    bot_pad_arr = np.full((height+pad_h,pad_w),255)
    image = np.append(image,bot_pad_arr,axis=1)
height,width = image.shape[0],image.shape[1]
first_c = 0
init_pointer = 0
second_c = 0
third_c = 0
end_pointer = 0
big_empty_chunk = 0
prev_cut = 0
row_num = 1
for i in range(0,height):
    prev_avg = 0
    counter = 0
    #print("row {} to {}".format(i,i+line_space))
    for j in range(0,width,line_space):
        chunk = image[i:i+line_space,j:j+line_space]
        s = chunk.sum()
        avg = s/(line_space**2)
        if prev_avg == 0:
            prev_avg = avg
        else:
            per_change = int((abs(avg-prev_avg)/prev_avg)*100)
            prev_avg = avg
            if per_change >= 0 and per_change <= 1:
                counter = counter + 1
    per_change = int((counter/(width/line_space))*100)
    if per_change >= 97:
        #print("this is an empty line")
        if(big_empty_chunk == 0):
            if(first_c == 0):
                first_c = 1
                init_pointer = (i,i+line_space)
            else:
                if(second_c == 0):
                    second_c = 1
                else:
                    if(third_c == 0):
                        third_c = 1
                        end_pointer = (i,i+line_space)
                    else:
                        big_empty_chunk = 1
                        #print("Found a chunk")
        else:
            if(big_empty_chunk != 0):
                end_pointer = (i,i+line_space)
    else:
        if(big_empty_chunk != 0):
            start = init_pointer[1]
            end = end_pointer[0]
            mid = int((start+end)/2)
            chunk_size = (mid-prev_cut)
            #print(chunk_size)
            image[mid:mid+(2),:] = 0
            cv2.imwrite("rows/row{}.png".format(row_num),image[prev_cut+2:mid,:])
            #print("row_num = {}".format(row_num))
            #print("{}:{} , {}:{}".format(prev_cut+2,mid,0,"er"))
            rows_list.append((prev_cut,mid))
            row_num = row_num + 1
            prev_cut = mid
            first_c = 0
            init_pointer = 0
            second_c = 0
            third_c = 0
            end_pointer = 0
            big_empty_chunk = 0
    cv2.imwrite("rows/row{}.png".format(row_num),image[prev_cut+2:,:]) 
#print("{}:{} , {}:{}".format(prev_cut+2,"ec",0,"er"))
rows_list.append((prev_cut,image.shape[0]))
   # print()
   # print()
        
cv2.imwrite("samp33.png",image)
print(rows_list)

In [28]:
rows_list

[(0, 63),
 (63, 454),
 (454, 509),
 (509, 547),
 (547, 584),
 (584, 631),
 (631, 727),
 (727, 791),
 (791, 836),
 (836, 927),
 (927, 990),
 (990, 1027),
 (1027, 1087),
 (1087, 1134),
 (1134, 1344),
 (1344, 1530)]

def runit(row_left,row_right,image_json,sno):
    #reading image to array
    image = cv2.imread("image3.png",0)
    if row_right == "er":
        image = image[row_left:,:]
    else:
        image = image[row_left:row_right,:]
    #print(image.shape)
    #converting image to data
    d = pytesseract.image_to_data(image, output_type=Output.DICT)
    #getting average font size of the document
    font = getFontSize(d)
    #print(font)
    #print(font)
    #calculating approx line space of the document
    line_space = int(1.4*font)
    #initialising the kernel
    kernel = getKernel(int(line_space))

    #adjusting the image shape to fit kernel sliding
    height,width = image.shape[0],image.shape[1]
    pad_h = line_space - (height % line_space)
    if pad_h > 1:
        pad_h_1 = int(pad_h/2)
        pad_h_2 = pad_h - pad_h_1
        top_pad_arr_1 = np.full((pad_h_1,width),255)
        top_pad_arr_2 = np.full((pad_h_2,width),255)
        image = np.append(top_pad_arr_1,image,axis=0)
        image = np.append(image,top_pad_arr_2,axis=0)
    else:
        top_pad_arr = np.full((pad_h,width),255)
        image = np.append(image,top_pad_arr,axis = 0)
    pad_w = line_space - (width % line_space)
    if pad_w > 1:
        pad_w_1 = int(pad_w/2)
        pad_w_2 = pad_w - pad_w_1
        bot_pad_arr_1 = np.full((height+pad_h,pad_w_1),255)
        bot_pad_arr_2 = np.full((height+pad_h,pad_w_2),255)
        image = np.append(bot_pad_arr_1,image,axis=1)
        image = np.append(image,bot_pad_arr_2,axis=1)
    else:
        bot_pad_arr = np.full((height+pad_h,pad_w),255)
        image = np.append(image,bot_pad_arr,axis=1)
    height,width = image.shape[0],image.shape[1]
    first_c = 0
    mid = 0
    init_pointer = 0
    second_c = 0
    third_c = 0
    end_pointer = 0
    big_empty_chunk = 0
    prev_cut = 0
    row_num = 1
    for i in range(0,width):
        prev_avg = 0
        counter = 0
        #print("col {} to {}".format(i,i+line_space))
        for j in range(0,height,line_space):
            chunk = image[j:j+line_space,i:i+line_space]
            #print(j,end=" : ")
            s = chunk.sum()
            #print("chunk sum = {}".format(s), end=" , ")
            avg = s/(line_space**2)
            #print("avg = {}".format(avg), end=" , ")
            #print("prev_avg = {}".format(prev_avg), end=" , ")
            if prev_avg == 0:
                prev_avg = avg
                prev_per_change = 0
                #print("total_per_change = {}".format(0))
            else:
                per_change = int((abs(avg-prev_avg)/prev_avg)*100)
                total_per_change = abs(prev_per_change-per_change)
                prev_avg = avg
                #print("total_per_change = {}".format(total_per_change))
                if per_change >= 0 and per_change <= 2:
                    counter = counter + 1
                prev_per_change = per_change
        #print("counter = {}".format(counter), end=" , ")
        #print("avg = {}".format(avg), end=" , ")
        per_change = int((counter/(height/line_space))*100)
        #print("total_per_change = {}".format(per_change))
        if per_change >= 60:
            #print("this is an empty line")
            if(big_empty_chunk == 0):
                if(first_c == 0):
                    first_c = 1
                    init_pointer = (i,i+line_space)
                else:
                    if(second_c == 0):
                        second_c = 1
                    else:
                        if(third_c == 0):
                            third_c = 1
                            end_pointer = (i,i+line_space)
                        else:
                            big_empty_chunk = 1
                            #print("Found a chunk")
            else:
                if(big_empty_chunk != 0):
                    end_pointer = (i,i+line_space)

        else:
            if(big_empty_chunk != 0):
                start = init_pointer[1]
                end = end_pointer[0]
                mid = int((start+end)/2)
                chunk_size = (mid-prev_cut)
                #print(chunk_size)
                image[:,mid:mid+(2)] = 0
                cv2.imwrite("rows/cols/row{}_col{}.png".format(ext,row_num),image[:,prev_cut:mid])
                image_json[sno] = {"coordinate":[row_left,row_right,prev_cut,mid]}
                print("mid = {}".format(mid))
                row_num = row_num + 1
                prev_cut = mid
                first_c = 0
                init_pointer = 0
                second_c = 0
                third_c = 0
                end_pointer = 0
                big_empty_chunk = 0
            
        cv2.imwrite("rows/cols/row{}_col{}.png".format(ext,row_num),image[:,prev_cut:])
    image_json[sno]={"coordinate":[row_left,row_right,prev_cut,"c"]}



In [19]:
def runit(row_left,row_right,image_json,sno):
    #reading image to array
    image = cv2.imread("image3.png",0)
    image = image[row_left:row_right,:]
    #print(image.shape)
    #converting image to data
    d = pytesseract.image_to_data(image, output_type=Output.DICT)
    #getting average font size of the document
    font = getFontSize(d)
    #print(font)
    #print(font)
    #calculating approx line space of the document
    line_space = int(2*font)
    #initialising the kernel
    #adjusting the image shape to fit kernel sliding
    height,width = image.shape[0],image.shape[1]
    first_c = 0
    mid = 0
    init_pointer = 0
    second_c = 0
    third_c = 0
    end_pointer = 0
    big_empty_chunk = 0
    prev_cut = 0
    row_num = 1
    for i in range(0,width-line_space):
        prev_avg = 0
        counter = 0
        #print("col {} to {}".format(i,i+line_space))
        for j in range(0,height-2,2):
            chunk = image[j:j+2,i:i+line_space]
            #print(j,end=" : ")
            s = chunk.sum()
            #print("chunk sum = {}".format(s), end=" , ")
            avg = s/(line_space*2)
            #print("avg = {}".format(avg), end=" , ")
            #print("prev_avg = {}".format(prev_avg), end=" , ")
            if prev_avg == 0:
                prev_avg = avg
                prev_per_change = 0
                #print("total_per_change = {}".format(0))
            else:
                per_change = int((abs(avg-prev_avg)/prev_avg)*100)
                total_per_change = abs(prev_per_change-per_change)
                prev_avg = avg
                #print("total_per_change = {}".format(total_per_change))
                if per_change >= 0 and per_change <= 2:
                    counter = counter + 1
                prev_per_change = per_change
        #print("counter = {}".format(counter), end=" , ")
        #print("avg = {}".format(avg), end=" , ")
        per_change = int((counter/(height/line_space))*100)
        #print("total_per_change = {}".format(per_change))
        if per_change >= 60:
            #print("this is an empty line")
            if(big_empty_chunk == 0):
                if(first_c == 0):
                    first_c = 1
                    init_pointer = (i,i+line_space)
                else:
                    if(second_c == 0):
                        second_c = 1
                    else:
                        if(third_c == 0):
                            third_c = 1
                            end_pointer = (i,i+line_space)
                        else:
                            big_empty_chunk = 1
                            #print("Found a chunk")
            else:
                if(big_empty_chunk != 0):
                    end_pointer = (i,i+line_space)

        else:
            if(big_empty_chunk != 0):
                start = init_pointer[1]
                end = end_pointer[0]
                mid = int((start+end)/2)
                chunk_size = (mid-prev_cut)
                #print(chunk_size)
                image[:,mid:mid+(2)] = 0
                cv2.imwrite("rows/cols/row{}_col{}.png".format(ext,row_num),image[:,prev_cut:mid])
                image_json[sno] = {"coordinate":[row_left,row_right,prev_cut,mid]}
                print("mid = {}".format(mid))
                row_num = row_num + 1
                prev_cut = mid
                first_c = 0
                init_pointer = 0
                second_c = 0
                third_c = 0
                end_pointer = 0
                big_empty_chunk = 0
            
        cv2.imwrite("rows/cols/row{}_col{}.png".format(ext,row_num),image[:,prev_cut:])
    image_json[sno]={"coordinate":[row_left,row_right,prev_cut,"c"]}



In [572]:
image_json = {}
for i in range(0,len(rows_list)):
    print(rows_list[i][0])
    print(rows_list[i][1])
    runit(rows_list[i][0],rows_list[i][1],image_json,i+1)

2
63
65
454
mid = 46
mid = 559
456
509
mid = 355
mid = 865
511
547
549
584
586
631
633
727
mid = 45
729
791
mid = 376
mid = 880
793
836
838
927
mid = 360
mid = 876
929
990
mid = 44
992
1027
1029
1087
mid = 50
mid = 223
1089
1134
mid = 46
1136
1344
1346
er


In [8]:
import cv2
import numpy as np

img = cv2.imread('iamge_test.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
invGamma = 1.0 / 0.3
table = np.array([((i / 255.0) ** invGamma) * 255
for i in np.arange(0, 256)]).astype("uint8")

# apply gamma correction using the lookup table
gray = cv2.LUT(gray, table)

ret,thresh1 = cv2.threshold(gray,80,255,cv2.THRESH_BINARY)

#thresh = cv2.adaptiveThreshold(gray,255,1,1,11,2)
contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

def biggestRectangle(contours):
    biggest = None
    max_area = 0
    indexReturn = -1
    for index in range(len(contours)):
            i = contours[index]
            area = cv2.contourArea(i)
            if area > 100:
                peri = cv2.arcLength(i,True)
                approx = cv2.approxPolyDP(i,0.1*peri,True)
                if area > max_area: #and len(approx)==4:
                        biggest = approx
                        max_area = area
                        indexReturn = index
    return indexReturn

indexReturn = biggestRectangle(contours)
hull = cv2.convexHull(contours[indexReturn])
cv2.imwrite('hola.png',cv2.drawContours(img, [hull], 0, (0,255,0),3))
#cv2.imwrite('hola.png',thresh1)


True

In [9]:
co_image = cv2.drawContours(img, [hull], 0, (0,255,0),3)

In [24]:
co_image[0:1,0:1][0]

array([[149,  95,  84]], dtype=uint8)

In [17]:
for i in hull:
    x = i[0][0]
    y = i[0][1]
    co_image[x:x+5,y:y+5] = ()

1144
1148
1152
1153
1154
1155
1155
1152
1149
1145
1127
1103
1097
1096
989
986
978
957
193
157
155
154
153
153
163
164
171
176
1104
1117
1131


In [25]:
import cv2
import numpy as np


# load image as grayscale
img = cv2.imread('iamge_test.jpg')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# threshold input image using otsu thresholding as mask and refine with morphology
ret, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU) 
kernel = np.ones((9,9), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

# put thresh into 
result = img.copy()
result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
result[:, :, 3] = mask

# save resulting masked image
cv2.imwrite('retina_masked.png', result)

True

In [33]:
from PIL import Image
im = Image.open('image_new.tiff')

In [34]:
print(im.info)

{'compression': 'raw', 'dpi': (1, 1), 'resolution': (1, 1)}
